* This notebook contains a CNN model trained on drowsy driving images. <br>
* 60s Time series were windowed from original win60s events. 
* These time series were then oversampled using the OSTSC package in R. 
* Oversampled time series were converted to MTF and RP images using the pyts package in pyton.

In [1]:
import pandas as pd
import numpy as np
import random
import os
from keras.utils import np_utils
from keras import layers
from keras import models
from keras import optimizers
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras import metrics
from keras import backend as K
from sklearn.utils import class_weight
from matplotlib import pyplot
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from keras.datasets import mnist
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

os.getcwd()




/Users/apple/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'/Users/apple/Drowsy/60'

In [2]:
#Read image array from disk
x_train = np.loadtxt('x_train_MTF_j.txt').reshape((3496,61,61)) #using MTF
y_train = np.loadtxt('y_train_MTF_j.txt')
print(x_train.shape)
print(y_train.shape)

(3496, 61, 61)
(3496,)


In [3]:
#Defining a bunch of custom functions for preprocessing

#Normalize the images
def norm(data_x):
#convert to float32 and normalize to [0,1]
    data_x = data_x.astype('float32')
    data_x /= np.amax(data_x)
    return data_x

# convert labels to integers
def l_i(data_y):
    encoder = LabelEncoder()
    encoder.fit(data_y)
    data_y = encoder.transform(data_y)
    return data_y

#convert greyscale image to a 3D image
def rgb(greyscale):
    '''Takes in a 1 channel image and converts to rgb'''
    rgb_batch = np.repeat(greyscale[..., np.newaxis], 3, -1)
    return rgb_batch

train_x = norm(x_train)
train_y = l_i(y_train)
train_x = rgb(x_train)
print('Shape of the training image tensor after preprocessing: ', train_x.shape)

Shape of the training image tensor after preprocessing:  (3496, 61, 61, 3)


In [5]:
############TRANSFER LEARNING##########
from keras.applications.vgg19 import VGG19 #Using VGG19
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.models import Model
from keras.optimizers import SGD

conv_base = VGG19(weights='imagenet', #Using pretrained weights of imagenet
                  include_top=False,
                  input_shape= False)
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In the step below we apply the convolutional base of the pretrained model on our training set of images.

In [6]:
train_features = conv_base.predict(train_x)
train_features = np.reshape(train_features, (3496, 1 * 1 * 512)) #flatten it
print(train_features.shape)

(3496, 512)


In [7]:
#defining 3 custom metric for guiding our neural network
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    #value, update_op = metrics.roc_auc_score(y_true, y_pred)
    value, update_op = tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value
    
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [9]:
#Adding layers to the pretrained conv base                
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=1 * 1 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

#Fine tuning
#conv_base.trainable = True
#set_trainable = False
#for layer in conv_base.layers:
#    if layer.name == 'block5_conv1':
#        set_trainable = True
 #   if set_trainable:
  #      layer.trainable = True
   # else:
    #    layer.trainable = False
        
#Compiling the model 
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=[auc_roc])



model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 131,585
Trainable params: 131,585
Non-trainable params: 0
_________________________________________________________________


In [10]:
#Fitting the model
history = model.fit(train_features, train_y,
                    epochs=40, steps_per_epoch = 50)

Epoch 1/40
50/50 [==============================] - 4s 77ms/step - loss: 0.7616 - auc_roc: 0.4987
Epoch 2/40
50/50 [==============================] - 3s 68ms/step - loss: 0.7203 - auc_roc: 0.5299
Epoch 3/40
50/50 [==============================] - 3s 66ms/step - loss: 0.6962 - auc_roc: 0.5510: 1s - loss: 0.6993 
Epoch 4/40
50/50 [==============================] - 3s 67ms/step - loss: 0.6811 - auc_roc: 0.5700: 0s - loss: 0.6814 - auc_roc: 0.56
Epoch 5/40
50/50 [==============================] - 3s 68ms/step - loss: 0.6689 - auc_roc: 0.5859
Epoch 6/40
50/50 [==============================] - 3s 68ms/step - loss: 0.6601 - auc_roc: 0.5984
Epoch 7/40
50/50 [==============================] - 4s 72ms/step - loss: 0.6522 - auc_roc: 0.6085
Epoch 8/40
50/50 [==============================] - 3s 69ms/step - loss: 0.6466 - auc_roc: 0.6167
Epoch 9/40
50/50 [==============================] - 3s 69ms/step - loss: 0.6414 - auc_roc: 0.6238: 1s - loss: 0.6421 - 
Epoch 10/40
50/50 [======================

In [11]:
#loading the test dataset
#image
x_test = np.loadtxt('x_test_MTF_j.txt').reshape((204,61,61))
x_test = norm(x_test)
x_test = rgb(x_test)
print('Shape of the test image tensor after preprocessing: ', x_test.shape)
#labels
y_test = np.loadtxt('y_test_MTF_j.txt')
y_test = l_i(y_test)
print('Shape of the test labels', y_test.shape)

Shape of the test image tensor after preprocessing:  (204, 61, 61, 3)
Shape of the test labels (204,)


In [12]:
#Fitting the conv base of the pretrained model on our test set
test_features = conv_base.predict(x_test)
test_features = np.reshape(test_features, (204, 1 * 1 * 512)) #flatten it
print(test_features.shape)

(204, 512)


In [41]:
loss, auc = model.evaluate(test_features, y_test)
print('AUC using the evaluate function is: ', auc)

204/204 [==============================] - 0s 96us/step
AUC using the evaluate function is:  0.7045168537719577


#### Note that the AUC obtained by pretrained VGG19 on MTF images is 70.45%

In [60]:
#predictions = model.predict(test_features) #array of prediction probabilities
roc_mtf_19 = np.column_stack((y_test, predictions))
roc_mtf_19_df = pd.DataFrame(roc_mtf_19)
roc_mtf_19_df.to_csv("roc_mtf_19_predicted.csv")

In [14]:
#Saving the model to disk

#from keras.models import model_from_json
#model_json = model.to_json()                    # serialize model to JSON
#with open("VGG19_MTF.json", "w") as json_file:
 #   json_file.write(model_json)
#model.save_weights("VGG19_MTF.h5")               #serialize weights to HDF5
#print("Saved model to disk")

Saved model to disk


In [15]:
# load json and create model

#json_file = open('VGG19_MTF.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
#loaded_model.load_weights("VGG19_MTF.h5")          # load weights into new model
#print("Loaded model from disk")

Loaded model from disk


In [16]:
# evaluate loaded model on test data

#loaded_model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
 #             loss='binary_crossentropy',
  #            metrics=[auc_roc])
#predictions_loaded = loaded_model.predict(test_features)
#print('AUC attained by the pretrained CNN model on MTF images: ', roc_auc_score(y_test, predictions_loaded))

AUC attained by the pretrained CNN model on MTF images:  0.6999739786625032


Now we repeat the entire process with images generated via RP

In [17]:
#Read image array from disk
x_train_RP = np.loadtxt('x_train_RP_j.txt').reshape((3496,61,61)) #using MTF
y_train_RP = np.loadtxt('y_train_RP_j.txt')
print('Original shape of training image tensor: ', x_train.shape)
print('Original shape of training labels: ', y_train.shape)

#Defining a bunch of custom functions for preprocessing

#Normalize the images
def norm(data_x):
#convert to float32 and normalize to [0,1]
    data_x = data_x.astype('float32')
    data_x /= np.amax(data_x)
    return data_x

# convert labels to integers
def l_i(data_y):
    encoder = LabelEncoder()
    encoder.fit(data_y)
    data_y = encoder.transform(data_y)
    return data_y

#convert greyscale image to a 3D image
def rgb(greyscale):
    '''Takes in a 1 channel image and converts to rgb'''
    rgb_batch = np.repeat(greyscale[..., np.newaxis], 3, -1)
    return rgb_batch

train_x_RP = norm(x_train_RP)
train_y_RP = l_i(y_train_RP)
train_x_RP = rgb(x_train_RP)
print('Shape of the training image tensor after preprocessing: ', train_x_RP.shape)

Original shape of training image tensor:  (3496, 61, 61)
Original shape of training labels:  (3496,)
Shape of the training image tensor after preprocessing:  (3496, 61, 61, 3)


In [18]:
train_features_RP = conv_base.predict(train_x_RP)
train_features_RP = np.reshape(train_features_RP, (3496, 1 * 1 * 512)) #flatten it
print(train_features_RP.shape)

(3496, 512)


In [20]:
#Adding layers to the pretrained conv base                
model_RP = models.Sequential()
model_RP.add(layers.Dense(256, activation='relu', input_dim=1 * 1 * 512))
model_RP.add(layers.Dropout(0.5))
model_RP.add(layers.Dense(1, activation='sigmoid'))

#Fine tuning
#conv_base.trainable = True
#set_trainable = False
#for layer in conv_base.layers:
#    if layer.name == 'block5_conv1':
#        set_trainable = True
 #   if set_trainable:
  #      layer.trainable = True
   # else:
    #    layer.trainable = False
        
#Compiling the model 
model_RP.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=[auc_roc])



model_RP.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 131,585
Trainable params: 131,585
Non-trainable params: 0
_________________________________________________________________


In [21]:
history_RP = model_RP.fit(train_features_RP, train_y_RP,
                    epochs=40, steps_per_epoch = 50)

Epoch 1/40
50/50 [==============================] - 4s 79ms/step - loss: 0.7408 - auc_roc: 0.5021
Epoch 2/40
50/50 [==============================] - 3s 69ms/step - loss: 0.7023 - auc_roc: 0.5488
Epoch 3/40
50/50 [==============================] - 3s 65ms/step - loss: 0.6772 - auc_roc: 0.5763
Epoch 4/40
50/50 [==============================] - 3s 65ms/step - loss: 0.6605 - auc_roc: 0.5986
Epoch 5/40
50/50 [==============================] - 3s 66ms/step - loss: 0.6493 - auc_roc: 0.6157
Epoch 6/40
50/50 [==============================] - 3s 66ms/step - loss: 0.6392 - auc_roc: 0.6292
Epoch 7/40
50/50 [==============================] - 3s 68ms/step - loss: 0.6310 - auc_roc: 0.6399
Epoch 8/40
50/50 [==============================] - 3s 67ms/step - loss: 0.6261 - auc_roc: 0.6487
Epoch 9/40
50/50 [==============================] - 3s 68ms/step - loss: 0.6197 - auc_roc: 0.6562: 2s - loss: 0.6204 -  - ETA: 1s - loss: 
Epoch 10/40
50/50 [==============================] - 3s 67ms/step - loss: 0.6

In [22]:
#loading the test dataset
#image
x_test_RP = np.loadtxt('x_test_RP_j.txt').reshape((204,61,61))
x_test_RP = norm(x_test_RP)
x_test_RP = rgb(x_test_RP)
print('Shape of the test image tensor after preprocessing: ', x_test_RP.shape)
#labels
y_test_RP = np.loadtxt('y_test_RP_j.txt')
y_test_RP = l_i(y_test_RP)
print('Shape of the test labels', y_test_RP.shape)

#Fitting the conv base of the pretrained model on our test set
test_features_RP = conv_base.predict(x_test_RP)
test_features_RP = np.reshape(test_features_RP, (204, 1 * 1 * 512)) #flatten it
print('Shape of test features tensor after fitting the conv base: ', test_features_RP.shape)

Shape of the test image tensor after preprocessing:  (204, 61, 61, 3)
Shape of the test labels (204,)
Shape of test features tensor after fitting the conv base:  (204, 512)


In [52]:
model_RP.metrics_names

['loss', 'auc_roc']

In [50]:
loss_RP, auc_RP = model_RP.evaluate(test_features_RP, y_test_RP)
print('AUC using the evaluate function is: ', auc_RP)

204/204 [==============================] - 0s 92us/step
AUC using the evaluate function is:  0.7337754929766935


#### Note that AUC obtained by pretrained VGG19 on RP images is 73.37%

In [61]:
#predictions_RP = model_RP.predict(test_features_RP) #array of prediction probabilities
roc_rp_19 = np.column_stack((y_test_RP, predictions_RP))
roc_rp_19_df = pd.DataFrame(roc_rp_19)
roc_rp_19_df.to_csv("roc_rp_19_predicted.csv")

In [36]:
#Saving the model to disk
#from keras.models import model_from_json
# serialize model to JSON
#model_json = model_RP.to_json()
#with open("VGG19_RP.json", "w") as json_file:
 #   json_file.write(model_json)
#serialize weights to HDF5
#model.save_weights("VGG19_RP.h5")
#print("Saved model to disk")

Saved model to disk


In [37]:
# load json and create model
#json_file = open('VGG19_RP.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
# load weights into new model
#loaded_model.load_weights("VGG19_RP.h5")
#print("Loaded model from disk")

Loaded model from disk


# Summary

#### In this notebook we used the convolutional base of an imagenet based pre-trained VGG19 network to generate features for both our training and test images. We then built a small model [FC - Dropout - FC] and used it to classify for our test images. 

### AUC obtained with MTF images : 70%
### AUC obtained with RP images : 73%

##### Training details: <br> <br> Optimize = RMSprop(lr=2e-5) <br> Loss = Binary Crossentropy <br> Training metric = AUC  <br> Epochs = 40 <br> Steps per epoch = 50
       

                    